<a href="https://colab.research.google.com/github/LeoAlvarengaDev/UC_Dual_CDL-1S_2023-UNA/blob/main/base_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Bibliotecas
import pandas as pd
import re
from pandas.core import apply
import numpy as np

In [57]:
# Definindo uma funação para realizar a criacão de data frames
def leitura_data(data_frame,index):
    '''
    O parâmetro data_frame especifica o nome ou caminho do arquivo Excel a ser lido.
    O parâmetro index determina a coluna do DataFrame que será utilizada como índice.
    '''
    df = pd.read_excel(data_frame,index_col=index)
    df = df.drop(2,axis=index)
    return df

In [58]:
# Definindo uma função para remolver os numeros e parenteses da coluna tipo
def limpa_tipo(data_frame,coluna):
    '''
    O parâmetro coluna especifica a coluna do DataFrame onde será feita a remoção.
    '''
    data_frame[coluna] = data_frame[coluna].str.replace('\d+|\(|\)','', regex=True)
    return data_frame

In [56]:
# Definindo uma função para extrair o endereço
def extrair_endereco(coluna):
    '''
    O parâmetro coluna especifica a coluna do DataFrame onde será feita a extração.
    '''
    padrao = r'\d{5}-\d{3}' # Expressão regular para encontrar o CEP
    match = re.search(padrao, coluna)
    if match is not None:
        cep = match.group()
        data_frame = coluna.split(cep)[0].strip() + cep# Separando o endereço e removendo espaços desnecessários
        return data_frame
    else:
        return ''

In [27]:
# Função para extrair os horarios de doimgo até sexta
def extrair_horarios(string,dia,proximo):
    if isinstance(string, str) and dia in string:
        horarios = string.split(dia)[1].split(proximo)[0].strip()
        return horarios
    else:
        return ''

# Função para extrair os horarios de sábado
def extrair_sabado(string):
    if isinstance(string, str) and 'sábado' in string:
        horarios = string.split('sábado')[1].strip()
        return horarios
    else:
        return ''

# Função utilizada para realizar o tratamento da coluna horarios
def separa_horarios(df):
  dias_semana = ['domingo','segunda-feira','terça-feira','quarta-feira','quinta-feira','sexta-feira','sábado']
  for i in range(len(dias_semana)):
    if i < 6 :
      df[dias_semana[i]] = df['horarios'].apply(lambda x: extrair_horarios(x, dias_semana[i],dias_semana[i+1]))
    else:
      df['sábado'] = df['horarios'].apply(extrair_sabado)
  return df

In [28]:
# Definindo um função para salvar os arquivos
def salva(df, nome):
  '''O parâmetro nome determina qual será o nome utilizado para salvar o arquivo'''
  return df.to_excel(nome)

In [77]:
# Função para extrair informações de um DataFrame
def extrair(df,coluna_df,pesquisa,nova_coluna,preechimento):
  '''O parâmetro coluna_df especifica a coluna do DataFrame onde será feita a busca.

  O parâmetro pesquisa determina o critério de busca para selecionar as linhas do DataFrame.

  O parâmetro nova_coluna indica a coluna do DataFrame onde os valores extraídos serão armazenados.

  O parâmetro preenchimento determina o valor a ser preenchido nas linhas selecionadas.'''

  df.loc[df[coluna_df].str.contains(pesquisa),nova_coluna] = preechimento
  return df

In [118]:
df = leitura_data('/content/drive/MyDrive/Tramento_Dados/Rua_Timbiras.xlsx','index')
df = limpa_tipo(df,'tipo')
df['endereco'] = df['endereco'].astype(str).apply(extrair_endereco)
df = separa_horarios(df)

In [119]:
df =  extrair(df,'endereco','Rua dos Timbiras','Rua','Rua dos Timbiras')

In [120]:
lista_bairro = ['Centro','Funcionários','Savassi','Boa Viagem']
for i, a in enumerate(lista_bairro):
  df = extrair(df,'endereco',lista_bairro[i],'Bairro',a)

In [121]:
df =  extrair(df,'endereco','Belo Horizonte','Cidade','Belo Horizonte')

In [122]:
list_cep = ['30130-170','30140-064','30140-060','30130-180','30140-061']
for  index, cep in enumerate(list_cep):
  df =  extrair(df,'endereco',cep,'CEP',cep)

In [123]:
salva(df,'Timbiras.xlsx')

In [ ]:
df